Link to another tutorial site that I was using to figure this out:
https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing #Convert categorical data to neumeric values

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [2]:
keep_columns=['lat','month','contest-tmp2m-14d__tmp2m',
              'climateregions__climateregion','lon']
X_columns=[['lat','month',
              'climateregions__climateregion','lon']]

In [3]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [4]:
def eliminate_columns(df):
    for col in df.columns:
        if col in keep_columns:
            print(col)
        else: 
            df=df.drop(col, axis=1)
    return df

In [5]:
#need to come up with a different way to do this, because it is not working
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [6]:
def normanlize_data(df,train_df):
    for column in df.columns:
        df[column] = df[column]  /  train_df[column].abs().max()
    return df

In [7]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [8]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [9]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [10]:
print('training set prep')
train_df=seperate_monthly(train_df)
#train_df=eliminate_columns(train_df)
print('test set prep')
test_df=seperate_monthly(test_df)
#test_df=eliminate_columns(test_df)

training set prep
test set prep


In [11]:
train_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
122229,122229,0.454545,0.566667,37.48,-4.49,-3.62,-6.73,-5.12,-5.61,-7.34,...,79.07,-26.04,33.00,-8.37,-2.30,25.20,8.97,-47.72,1,15
121119,121119,0.454545,0.500000,23.68,-3.98,-2.64,-3.65,-1.41,-1.11,0.45,...,-43.45,-24.92,-64.44,-83.76,29.97,-21.11,23.37,-3.49,1,16
78488,78488,0.363636,0.400000,704.17,20.45,19.97,19.77,16.30,15.15,15.82,...,9.88,23.39,-22.48,-12.41,-32.33,-10.19,20.61,-42.22,5,15
134028,134028,0.500000,0.166667,341.24,9.52,9.19,8.36,8.65,7.16,-0.94,...,5.69,32.58,-20.69,-0.84,-9.65,24.76,-36.97,53.33,5,15
133835,133835,0.500000,0.166667,231.83,12.93,13.38,11.70,14.33,10.50,7.65,...,15.64,44.21,3.04,11.70,-25.95,-8.89,11.09,-1.93,11,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133706,133706,0.500000,0.133333,859.49,20.72,20.91,20.91,20.60,17.32,19.04,...,-9.58,24.79,-21.00,-41.96,-28.69,21.33,4.83,14.24,6,16
128783,128783,0.454545,0.866667,103.80,2.52,6.96,-0.26,5.56,2.37,5.30,...,-19.42,34.64,-8.91,-33.77,-39.55,30.34,14.60,-34.87,1,15
149557,149557,0.500000,0.866667,214.62,14.96,18.34,16.00,16.24,13.97,15.35,...,30.60,26.00,-72.53,0.45,59.89,12.98,10.98,25.96,11,15
189033,189033,0.590909,0.700000,92.97,8.87,11.86,9.85,11.57,12.21,11.04,...,27.92,30.39,-44.38,-5.16,62.69,-1.41,4.65,50.69,11,15


In [12]:
test_df

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20,month,year
0,375734,0.0,0.833333,339.88,30.88,30.92,29.17,31.02,29.47,30.93,...,-29.25,40.88,-8.31,14.91,-24.62,31.05,-23.69,6.27,11,22
1,375735,0.0,0.833333,334.63,30.88,30.92,29.17,31.02,29.47,30.93,...,-28.62,45.37,-5.42,16.97,-23.94,28.84,-20.61,14.16,11,22
2,375736,0.0,0.833333,337.83,30.88,30.92,29.17,31.02,29.47,30.93,...,-27.67,49.76,-1.31,21.44,-19.06,26.85,-16.78,13.42,11,22
3,375737,0.0,0.833333,345.81,30.88,30.92,29.17,31.02,29.47,30.93,...,-19.32,52.62,-0.44,21.65,-23.12,23.70,-18.62,10.69,11,22
4,375738,0.0,0.833333,357.39,30.88,30.92,29.17,31.02,29.47,30.93,...,-9.89,51.23,-7.57,19.86,-30.56,20.66,-25.08,19.64,11,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31349,407083,1.0,0.866667,62.72,4.60,8.71,6.05,10.08,6.39,8.42,...,32.39,38.82,7.42,11.75,-23.62,-0.24,-5.94,51.23,12,22
31350,407084,1.0,0.866667,73.41,4.60,8.71,6.05,10.08,6.39,8.42,...,26.23,37.64,13.01,17.84,-22.05,-3.03,1.31,51.45,12,22
31351,407085,1.0,0.866667,70.00,4.60,8.71,6.05,10.08,6.39,8.42,...,21.06,36.53,14.15,23.12,-25.60,-5.88,9.32,45.32,12,22
31352,407086,1.0,0.866667,79.81,4.60,8.71,6.05,10.08,6.39,8.42,...,20.42,36.05,6.38,29.00,-27.06,-1.42,16.06,31.88,12,22


In [13]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [14]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [15]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [16]:
X_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352604 entries, 122229 to 141732
Columns: 246 entries, index to year
dtypes: float64(246)
memory usage: 664.5 MB


# Model

In [17]:
NN_model = Sequential()

#I might need to change the input_dim to give the actual dimentions
#Instead of being just 1
# The Input Layer :
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = 246, activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               63232     
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 512)               262656    
                                                                 
 dense_6 (Dense)             (None, 1)                 5

2023-02-22 17:51:44.921526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64::/opt/conda/lib
2023-02-22 17:51:44.921573: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [18]:
checkpoint_name = 'Weights--.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [19]:
NN_model.fit(X_train_df, y_train_df, epochs=10, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/10
8814/8816 [============================>.] - ETA: 0s - loss: 1.0943 - mean_absolute_error: 1.0943
Epoch 1: val_loss improved from inf to 1.22533, saving model to Weights--.hdf5
8816/8816 [==============================] - 88s 10ms/step - loss: 1.0942 - mean_absolute_error: 1.0942 - val_loss: 1.2253 - val_mean_absolute_error: 1.2253
Epoch 2/10
8815/8816 [============================>.] - ETA: 0s - loss: 0.8146 - mean_absolute_error: 0.8146
Epoch 2: val_loss improved from 1.22533 to 0.99930, saving model to Weights--.hdf5
8816/8816 [==============================] - 90s 10ms/step - loss: 0.8146 - mean_absolute_error: 0.8146 - val_loss: 0.9993 - val_mean_absolute_error: 0.9993
Epoch 3/10
8813/8816 [============================>.] - ETA: 0s - loss: 0.7250 - mean_absolute_error: 0.7250
Epoch 3: val_loss improved from 0.99930 to 0.70683, saving model to Weights--.hdf5
8816/8816 [==============================] - 91s 10ms/step - loss: 0.7250 - mean_absolute_error: 0.7250 - val_loss

In [20]:
# Load wights file of the best model :
wights_file = f'Weights--.hdf5'  # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [21]:
predictions = NN_model.predict(test_df)

980/980 [==============================] - 3s 3ms/step


# Saving Dataset in correct format

In [22]:
predictions

array([[24.99737  ],
       [25.039225 ],
       [25.037989 ],
       ...,
       [ 2.1388893],
       [ 2.3171253],
       [ 1.9024591]], dtype=float32)

In [23]:
final=pd.DataFrame(predictions,columns=['contest-tmp2m-14d__tmp2m'])

In [24]:
i=375734
final['index']=0
while i<407088:
    final['index'][i-375734]=i
    i+=1
final

,contest-tmp2m-14d__tmp2m,index
0,24.997370,375734
1,25.039225,375735
2,25.037989,375736
3,25.043791,375737
4,25.154512,375738
...,...,...
31349,3.178327,407083
31350,2.715512,407084
31351,2.138889,407085
31352,2.317125,407086


In [25]:
final.to_csv('/kaggle/working/wetherprediction.csv',index=False)